# Entrega 2, Grupo 02 - Aprendizaje Bayesiano

- Santiago Alaniz,  5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone,  4914555-0, bruno.de.simone@fing.edu.uy
- María Usuca,      4891124-3, maria.usuca@fing.edu.uy

## Objetivo

Implementar un algoritmo de predicción de palabras utilizando Naive Bayes. El modelo se entrenará con datos de conversaciones de WhatsApp y se probará en un simulador de cliente. 

El algoritmo debe ser capaz de recomendar palabras basadas en las últimas N palabras ingresadas en una frase, donde N es un hiperparámetro que se variará para evaluar el desempeño del modelo. Además, el modelo se reentrenará al finalizar cada frase para adaptarse a nueva evidencia. 

La implementación debe ser eficiente en términos de uso de CPU, utilizando las estructuras de datos más adecuadas en Python.


## Diseño

### Carga del dump de conversaciones en un grupo de WhatsApp

En la letra del laboratorio se menciona que se puede acceder al dump en formato `.csv` pero esto es un error en la letra. *WhatsApp* exporta las conversaciones en formato `.txt`.

Esto complejiza la carga de los datos ya que no podemos asumir que cada linea del archivo es una entrada del log.

Detectamos el siguiente patron en el archivo de texto

```
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
    ***
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
```

Nos valemos del modulo `src.regex` para definir la expresion regular que detecta el patron y extrae los mensajes (`LOG_ENTRY_PATTERN`).

`LOG_ENTRY_PATTERN = f'{metadata_pattern}{message_pattern}(?=\n{metadata_pattern}|$)':`

Esta expresion regular captura el contenido de cada entrada del log en dos grupos: `metadata` y `message`. 

- El grupo `metadata` captura la fecha y hora de la entrada, el nombre del usuario o el numero de telefono. 
- El grupo `message` captura el mensaje en si mismo y es el que nos interesa recuperar.

In [ ]:
import re
from src.whatsapp_regex import LOG_ENTRY_PATTERN
import random

SEED = 42069
FILE_PATH = './assets/chat.txt'
PATTERN = LOG_ENTRY_PATTERN

random.seed(SEED)

with open(FILE_PATH, 'r', encoding='utf-8') as file:
  data = file.read()

matches = re.findall(PATTERN, data)
data = [ match[1] for match in matches ]

data[:10]

### Preprocesamiento de datos

Para un algoritmo de sugerencia de palabras basado en aprendizaje bayesiano, el preprocesamiento de los datos es crucial para obtener resultados significativos. En nuestro caso, se compone de los siguientes pasos:

- **Tokenización ordenada**: Para que el algoritmo pueda identificar correctamente las palabras y su posición en la oración, es necesario dividir la oración en [tokens](https://es.wikipedia.org/wiki/Token).
- **Conversión a minúsculas**: Para que el algoritmo no trate las palabras como diferentes solo debido a las diferencias de mayúsculas y minúsculas.
- **Eliminación de caracteres especiales, numericos, emojis**: Estos componentes lexicos y semanticos no aportan información relevante para el modelo que busca sugerir palabras.
- **Relajar tildes con [unidecode](https://pypi.org/project/Unidecode/)**: Para que el modelo no distinga palabras con tildes de palabras sin tildes. Teniendo en cuenta que en los mensajes de WhatsApp es muy común que se omitan las tildes.
- **Validacion de palabras en espanol con un [corpus](http://universal.elra.info/product_info.php?cPath=42_43&products_id=1509)**: Para descartar palabras que no pertenecen al idioma español, chequeamos su existencia en un corpus. Esto es importante para evitar que el modelo sugiera palabras en otros idiomas o palabras que no existen.

Construimos la clase `src.G02Preprocessor` para encapsular el preprocesamiento de los datos y realizar los pasos mencionados anteriormente con mayor facilidad y control.

``` python
class G02Preprocessor:
    def __init__(self):
        self.V_SPA = set([unidecode(word.lower()) for word in cess_esp.words()])


    def apply(self, data, data_test=True):
        preprocessed_data = []

        for message in data:
            words = word_tokenize(message, language='spanish')
            words = [unidecode(word.lower()) for word in words]
            words = [word for word in words if word.isalpha()]
            if not words: continue
            if data_test:
                words = [word for word in words if word in self.V_SPA]
            if not words: continue

            preprocessed_data.append(words)

        return preprocessed_data
```


In [ ]:
from src.preprocessing import G02Preprocessor


preprocessor = G02Preprocessor()
ejemplo = random.choice(data)


print(f'\
Palabras del diccionario cess_esp: {len(preprocessor.V_SPA)} \n\
Frase de ejemplo: \n {ejemplo} \n\
Frase de ejemplo preprocesada: {preprocessor.apply([ejemplo])} \n'
)


### Particionamiento de datos

Antes de continuar, separaremos el conjunto de datos para luego poder entrenar y evaluar.

In [ ]:
from sklearn.model_selection import train_test_split

SEED_NUMBER = 342
TEST_SIZE   = 0.4
DEVEL_SIZE  = 0.5

train, test = train_test_split(data, test_size= TEST_SIZE, random_state= SEED_NUMBER)
test, devel = train_test_split(test, test_size= DEVEL_SIZE, random_state= SEED_NUMBER)

print (train)
print (test)
print (devel)

### Algoritmo

El algoritmo a implementar es un [clasificador bayesiano naive](https://es.wikipedia.org/wiki/Clasificador_bayesiano_ingenuo). Con la incorporación de un meta-pararametro `N`, que indica la cantidad de palabras anteriores a considerar para la predicción de la siguiente palabra. 

En la letra del laboratorio se menciona que este algoritmo es intensivo en CPU, es parte de la consigna del laboratorio implementar el algoritmo de la manera más eficiente posible.

Detectamos las siguientes tareas para cumplir con la consigna:

- **Definir las estructuras de datos**: Debemos diseñar las estructuras de datos que nos permitan almacenar la información necesaria para el funcionamiento del algoritmo. De forma tal que el acceso a la información y los calculos sean lo más eficientes posibles.
- **Modulo auxiliar `naive_bayes_utils.py`**: Una vez diseñadas las estructuras de datos, debemos implementarlas, proponemos hacerlo siguiendo un enfoque modular, donde se plasmen las ideas discutidas en el punto anterior.
- **Clase `G02NaiveBayesClassifier`**: Una vez implementadas las estructuras de datos, debemos implementar la clase `G02NaiveBayesClassifier` que encapsula el algoritmo de clasificación y mantiene todo el aparato auxiliar encapsulado bajo la misma clase. Esta clase esta definida en el modulo `bayesian_learning.py`.


#### Definir las estructuras de datos

Nuestro modelo busca sugerir la palabra más probable dado un contexto de N palabras anteriores. Esto es coincidente con el objetivo mas general de los clasificadores bayesianos, que es encontrar la hipótesis más probable dado un conjunto de evidencias. (`h_map`)

- `h_map` = argmax<sub>h</sub> P(h|D) = argmax<sub>h</sub> P(D|h) P(h) / P(D)

Los clasificadores bayesianos naive asumen que las evidencias son independientes entre si, por lo que la probabilidad de la hipótesis MAP se puede expresar como el producto de las probabilidades de cada evidencia dada la hipótesis multiplicado por la probabilidad de la hipótesis.

- `h_map` = argmax<sub>h</sub> P(D|h) P(h) / P(D) = argmax<sub>h</sub> P(D|h) P(h) = argmax<sub>h</sub> P(d<sub>1</sub>|h) ... P(d<sub>n</sub>|h) P(h)

Nuestro meta-parámetro `N` indica la cantidad de palabras anteriores a considerar para la predicción de la siguiente palabra. Es decir, `N` es la cantidad de evidencias que consideramos para calcular la probabilidad de la hipótesis (la palabra a sugerir).

Si bien un modelo bayesiano naive simplifica los calculos, diseñar estructuras de datos y algoritmos eficientes para calcular las probabilidades no es trivial. Proponemos las siguientes estructuras de datos para almacenar la información necesaria para el funcionamiento del algoritmo:

- `V`: Vocabulario de palabras en el conjunto de entrenamiento.
- `F_h`: Frecuencia de cada hipótesis (palabra) en el conjunto de entrenamiento.
- `F_hD`: Frecuencia de cada evidencia (palabra anterior) dado una hipótesis (palabra) en el conjunto de entrenamiento. A diferencia de `F_h`, `F_hD` es un diccionario de diccionarios y es afectado por el meta-parámetro `N`, ya que la cantidad de evidencias a considerar depende de `N`. Por ejemplo, si `N=4` y la hipótesis es `h`, `F_hD[h]` es un diccionario que contiene la frecuencia de cada combinación de 4 palabras anteriores a `h` en el conjunto de entrenamiento.

Mantener las frecuencias es computacionalmente hablando mas eficiente que mantener las probabilidades. Se plantea un esquema de actualizacion y uso a 'demanda', donde las prrobabilidades de h_map se calculan 'on the fly'.

*Nota*:

El enfoque de construccion de estas estructuras es greedy en el sentido de que si la frase desafortunadamente no tiene el largo de palabras necesario toma hasta donde puede. Por ejemplo, si `N=4` y la frase es `['hola', 'como', 'estas']`, el modelo solo considera las palabras `['hola', 'como']` para la prediccion de la siguiente palabra. Esto es un trade-off entre eficiencia y exactitud.

#### Modulo auxiliar `naive_bayes_utils.py`

El modulo `naive_bayes_utils.py` encapsula las estructuras de datos y los algoritmos auxiliares necesarios para el funcionamiento del clasificador bayesiano naive. 

Estas estructuras de datos fueron definidas en el punto anterior y fueron implementadas en el modulo `naive_bayes_utils.py` de la siguiente manera:

- `build(data, N)`: Genera contadores para las frecuencias de palabras individuales (`F_h`) y pares de palabras (`F_hD`) en un conjunto de datos de texto. Devuelve el número total de palabras (`V`), junto con los contadores.
- `p_h(h, V, F_h, data)`: Calcula una probabilidad ajustada para la ocurrencia de una palabra específica (`h`) en el conjunto de datos.
- `p_hD(d, h, V_SPA, F_h, F_hD, m=1)`: Calcula la probabilidad condicional de que una palabra (`d`) preceda a otra (`h`) usando m-stimador. 

*Nota*: 

En la firma de `p_h` abusamos de la notacion, enralidad estamos calculando la metrica tf-idf. Consideramos que esta metrica es mucho mas significativa que simplemente la frecuencia de la hipotesis en el conjunto de entrenamiento. [TF-IDF](https://es.wikipedia.org/wiki/Tf-idf) es una métrica comúnmente usada en el procesamiento del lenguaje natural para valorar la importancia de una palabra en un corpus en relación con su frecuencia en documentos específicos.

En la implementacion de `p_hD` se utiliza un m-stimador para evitar que la probabilidad condicional sea cero. Asumimos equiprobabilidad a priori de la evidencia, siendo esta 1/len(V_SPA) donde V_SPA es el vocabulario de palabras en español que utilizamos en el preprocesamiento de los datos. [cess_esp](http://universal.elra.info/product_info.php?cPath=42_43&products_id=1509)

In [ ]:
from src.naive_bayes_utils import *

V, F_h, F_hD = build(preprocessor.apply(train), N=4)

results = {
  'V': V,
  'F_h': F_h,
  'F_hD': F_hD,
}

for key, value in results.items():
  print(f'{key}: {value}')


#### Clase `G02NaiveBayesClassifier`

La clase `G02NaiveBayesClassifier` es nuestra implementación del algoritmo de clasificación Naive Bayes con un enfoque en el procesamiento del lenguaje natural. Estos son los métodos principales de la clase:

##### `__init__(self, data, N=1, M=1)`

Inicializa el clasificador:

- `data`: El conjunto de datos sobre el cual se construirá el modelo.
- `N`: El tamaño del contexto para el modelo.
- `M`: Un parámetro para suavizado.
  
Se realiza la preprocesamiento de los datos y se construyen las estadísticas (frecuencias de términos y pares de términos) utilizando la función `build`.

##### `predict(self, sentence)`

Realiza una predicción para una oración dada:

- `sentence`: La oración a clasificar.

Calcula el logaritmo de las probabilidades de cada palabra en el vocabulario dada la oración y selecciona la palabra con la mayor log-probabilidad que no esté ya en la oración. Somos conscientes que esta ultima modificacion no es ortodoxa, pero nos parecio una buena que el modelo no sugiera palabras que ya estan en la oracion.

##### `update(self, new_sentence)`

Actualiza el modelo con una nueva oración:

- `new_sentence`: La nueva oración para actualizar el modelo.

Realiza preprocesamiento en la nueva oración y actualiza las estadísticas del modelo (`F_h` y `F_hD`). También actualiza el conjunto de vocabulario y el número total de términos.

La clase también utiliza el preprocesador (`G02Preprocessor`) definido en el punto anterior para realizar el preprocesamiento de los datos.

*Nota*:

La decision de actualizar el Vocabulario en Español (`V_SPA`) con las palabras de la nueva oracion es discutible. Por un lado, es posible que la nueva oracion contenga palabras que no pertenecen al idioma español, como tambien que el usuario escriba palabras del espanol poco usuales y que estas no esten dentro del compendio del `cess_esp`. 

Por eso, asumimos que el usuario del cliente escribe en español (aunque se tome libertades a la hora de asignar tildes)

El siguiente codigo provee un benchmarking de la implementacion del algoritmo de clasificacion bayesiana naive.


In [ ]:
from src.bayesian_learning import G02NaiveBayesClassifier
import time


start = time.time()
clf = G02NaiveBayesClassifier(train, N=1)
print(f'[INIT]  Tiempo de ejecución: {time.time() - start:.4f}s')
test = ['horarios']

print(' '.join(test))

start = time.time()
for i in range(2): test.append(clf.predict(test))
print(f'[PREDICT] Tiempo de ejecución: {time.time() - start:.4f}s')

print(f'        V: {clf.V_DATA} \
    \n          F_h[{test[1]}]: {clf.F_h[test[1]]} \
    \n          F_hD[{test[1]}][{test[0]}]: {clf.F_hD[test[1]][test[0]]} \
    \n          F_h[{test[2]}]: {clf.F_h[test[2]]} \
    \n          F_hD[{test[2]}][{test[1]}]: {clf.F_hD[test[2]][test[1]]}')

start = time.time()

print(' '.join(test))

clf.update(' '.join(test))
print(f'[UPDATE]  Tiempo de ejecución: {time.time() - start:.4f}s')

print(f'        V: {clf.V_DATA} \
    \n          F_h[{test[1]}]: {clf.F_h[test[1]]} \
    \n          F_hD[{test[1]}][{test[0]}]: {clf.F_hD[test[1]][test[0]]} \
    \n          F_h[{test[2]}]: {clf.F_h[test[2]]} \
    \n          F_hD[{test[2]}][{test[1]}]: {clf.F_hD[test[2]][test[1]]}')

### Evaluación

En cualquier laboratorio o proyecto de investigación, la sección de evaluación es crucial para entender cómo se mide el rendimiento del modelo y qué métricas son relevantes para el problema en cuestión. Sin embargo, la naturaleza planteada en la letra del laboratorio no permite una evaluación cuantitativa del modelo. Por lo tanto, proponemos una evaluación cualitativa del modelo.

No sabemos como encapsular el hecho de que el modelo esta "sugeriendo bien" bajo una de las metricas presentada en la metodologia. Dado que es una percepción subjetiva del usuario que esta manteniendo una conversación con el modelo.

Lo que si podemos hacer, adoptando una evaluacion ya mas relajada. Dada una sesion de cliente, cuantificar la cantidad de veces que el modelo sugiere una palabra que el usuario considera correcta.

Alterando el tamaño de la ventana de palabras `N`  buscamos encontrar un equilibrio entre las siguientes propiedades deseables:

- **Memoria**: El modelo debe ser capaz de sugerir palabras que tengan coherencia con el grupo de Whatsapp de donde se extrajeron los datos.

- **Adaptacion**: El modelo debe ser capaz de adaptarse a la nueva evidencia, y mejorar su performance, sugiriendo palabras que tengan coherencia con la sesion de chat que se esta simulando.

Esta particularidad en la evaluación del modelo, tambien impacta en la definición de los conjuntos de entrenamiento, prueba y evaluación.

El clasificador bayesiano se construye a partir de un conjunto de entrenamiento. En nuestro caso, el conjunto de entrenamiento es el conjunto de mensajes de un grupo de WhatsApp. No hay conjunto de prueba ni conjunto de evaluación. El modelo se evalúa en tiempo real, mientras el usuario (en primera instancia nosotros y luego el cuerpo docente) interactúa con el modelo y evalúa la calidad de las sugerencias.

In [ ]:
from src.bayesian_learning import G02NaiveBayesClassifier  
N = 4
M = 1

devel = preprocessor.apply(devel)
clf = G02NaiveBayesClassifier(train, N=N, M=M)

- Falta jugar con N, hacerlo para todo devel? update?
- Armar una frase que no este en el modelo e ir probando cuanto demora en aprenderla

In [ ]:
i = 7 #28

palabras_acertadas = 0
frase = []
frase_sugerida = []
palabra_sugerida = devel[i][0]

for word in devel[i]:
    if (word == palabra_sugerida): palabras_acertadas+=1    
    frase.append(word)
    frase_sugerida.append(palabra_sugerida)
    palabra_sugerida = clf.predict(frase)

print("Frase original: "+ " ".join(frase))
print("Frase sugerida: "+ " ".join(frase_sugerida))
print("Cantidad de palabras acertadas: ", palabras_acertadas-1)

## Experimentación

### Simulador de cliente.

El experimento consiste en evaluar los distintos hiperparametros del modelo.

El siguiente script permite simular el comportamiento de un cliente que escribe frases, fue proporcionado por el cuerpo docente. Es de utilidad para experimentar con modelo, ya que permite ingresar frases y ver las sugerencias que el modelo realiza.

**Nota:**

El comportamiento del siguiente script en `VSCode` es diferente al comportamiento en la terminal. En VSCode, el script omite entradas, se desfasa y hace que el analisis de los resultados sea dificil. Por lo tanto, recomendamos ejecutar el script en la terminal. (`client.py`)

## Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?